In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [4]:
import pickle
import io
import torch
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else: return super().find_class(module, name)
sum_net_normal = torch.load("objects/sum_net_normal_{seed}.pkl".format(seed=3), map_location=device)

In [2]:
from sbi.inference.snle.snle_a import SNLE_A

from sbi.inference.base import *

import sbi.utils as utils
from sbi.utils.get_nn_models import *
from sbi.utils.sbiutils import *
from sbi.utils.torchutils import *
import torch
import numpy as np
import tqdm
import matplotlib.pyplot as plt
from torch import nn
import seaborn as sns
import random
import pickle
from pyknos.nflows import flows

from simulators.ricker import ricker
from networks.summary_nets import RickerSummary

from pyknos.nflows import distributions as distributions_
from pyknos.nflows import flows, transforms
from pyknos.nflows.nn import nets
from pyknos.nflows import *

In [11]:
import pickle
import io
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else: return super().find_class(module, name)

In [5]:
def magnitude(obs):
    obs_cont = torch.clone(obs).to(obs.device)
    for i in range(obs.shape[0]):
        obs_cont[i] += torch.abs(torch.randn(100).to(obs.device) * 15)

    # obs_cont = torch.tensor(obs_cont)
    return obs_cont

In [21]:
sum_net_normal = torch.load("objects/sum_net_normal_{seed}.pkl".format(seed=3), map_location=device)

In [6]:
low = torch.tensor([2., 0.]).to(device)
high = torch.tensor([8., 20.]).to(device)
prior = torch.distributions.uniform.Uniform(low, high)

num_simulations = 2000

In [7]:
x_summary = np.load("x_summary.npy")
theta_combine = np.load("theta_combine.npy")

In [5]:
x = torch.tensor(np.load("data/x.npy")).to(device)
theta = torch.tensor(np.load("data/theta.npy")).to(device)
x_cont = magnitude(x)

theta_combine = torch.cat([theta,theta]).detach().numpy()
x_combine = torch.cat([x, x_cont])
x_summary = sum_net_normal(x_combine)[1].detach().numpy()
np.save("x_summary.npy", x_summary)
np.save("theta_combine.npy", theta_combine)

NameError: name 'sum_net_normal' is not defined

In [8]:
theta_combine = torch.tensor(theta_combine)
x_summary = torch.tensor(x_summary)

In [16]:
print(theta_combine.shape)

torch.Size([4000, 2])


In [9]:
# Our distance minimization method
theta_combine = theta_combine.to(device)
x_summary = x_summary.to(device)
inference_robust = sbi.inference.snle.snle_a.SNLE_A(prior=prior, device='cpu')
density_estimator_robust = inference_robust.append_simulations(theta_combine, x_summary).train()
# posterior_robust = inference_robust.build_posterior(density_estimator_robust)

 Neural network successfully converged after 172 epochs.

In [15]:
theta_gt = np.array([[4, 10]])

obs = ricker(theta_gt).to(device)
obs_cont = magnitude(obs)

obs = sum_net_normal(obs)[1].reshape(1, 4)
obs_cont = sum_net_normal(obs_cont)[1]
print(obs.shape)

posterior_rob = inference_robust.build_posterior(density_estimator_robust)

posterior_rob.sample((10,), x=obs, num_chains=20, method="slice_np_vectorized")

torch.Size([1, 4])


RuntimeError: The size of tensor a (10000) must match the size of tensor b (2) at non-singleton dimension 1

In [29]:
theta_gt = np.array([[4, 10]])

obs = ricker(theta_gt).to(device)
obs_cont = magnitude(obs)

obs = sum_net_normal(obs)[1]
obs_cont = sum_net_normal(obs_cont)[1]
print(obs.shape)


post_samples_robust = posterior_robust.sample((10000,), x=obs.reshape(4))
post_samples_cont_robust = posterior_robust.sample((10000,), x=obs_cont.reshape(4))

plt.subplot(1, 2, 1)
sns.distplot(post_samples_robust[:, 0].cpu().numpy(), color='blue', kde=False, norm_hist=True, label="clean")
sns.distplot(post_samples_cont_robust[:, 0].cpu().numpy(), color="red", kde=False, norm_hist=True, label="misspecified")

plt.xlim(2, 8)
plt.ylabel("Density")
plt.xlabel("theta 1")
plt.title("Robust NPE - theta 1")
plt.axvline(x=theta_gt[0][0],ls="-",c="black")
plt.legend()

plt.subplot(1, 2, 2)
sns.distplot(post_samples_robust[:, 1].cpu().numpy(), color='blue', kde=False, norm_hist=True, label="clean")
sns.distplot(post_samples_cont_robust[:, 1].cpu().numpy(), color="red", kde=False, norm_hist=True, label="misspecified")

plt.xlim(0, 20)
plt.xlabel("theta 2")
plt.title("Robust NPE - theta 2")
plt.axvline(x=theta_gt[0][1],ls="-",c="black")

plt.legend()
plt.show()

torch.Size([1, 4])


RuntimeError: The size of tensor a (10000) must match the size of tensor b (2) at non-singleton dimension 1